<a href="https://colab.research.google.com/github/njainds/Colab_notebooks/blob/master/CCP_ICD10_Searchengine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import string
import time
import random
import pandas as pd
import numpy as np
import gc
import re
import torch
from torchtext import data
import spacy
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm
import zipfile
import gzip

tqdm.pandas(desc='Progress')
from collections import Counter
from textblob import TextBlob
from nltk import word_tokenize
import torch
import torch.nn as nn
import random
import keras
import numpy as np
from keras.layers import Conv1D, Conv2D
from keras import optimizers
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import nltk

# cross validation and metrics
from  sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection,linear_model,metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from torch.optim.optimizer import Optimizer
from unidecode import unidecode
import tensorflow as tf

In [5]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/CCP-ICDsearch"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
wikipedia-pubmed-and-PMC-w2v.bin


In [6]:
%%time
from gensim.models.keyedvectors import KeyedVectors

model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/CCP-ICDsearch/wikipedia-pubmed-and-PMC-w2v.bin', binary=True)
#model.save_word2vec_format('/content/drive/My Drive/CCP-ICDsearch/wikipedia-pubmed-and-PMC-w2v.txt', binary=False)

CPU times: user 1min 11s, sys: 7.73 s, total: 1min 19s
Wall time: 3min 48s


In [0]:
file='/content/drive/My Drive/CCP-ICDsearch/wikipedia-pubmed-and-PMC-w2v.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')[:200]
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file)) 

In [77]:
#model.wv['sequela']
#'aiwai' in model.wv.vocab

a=np.dot(model.wv['upper'],model.wv['upper'])
b=np.dot(model.wv['neoplasm'],model.wv['respiratory'])
print(a)
print(b)

8.417077
0.9276622


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until
